In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tifffile as tiff
from PIL import Image as PILImage

In [2]:
PILImage.MAX_IMAGE_PIXELS = 9999999999999

In [ ]:
""" img = PILImage.open("patient_100/patient_100_node_3.tif")
print(img.size)
img.thumbnail((1024,1024))
plt.imshow(np.array(img) / 255.0) """

In [4]:
def patch_image(image_path, patch_size=224, x1=0, y1=0, step_size=0):
    np_img = tiff.imread(image_path)
    y2 = y1 + step_size
    x2 = x1 + step_size
    np_img = np_img[y1:y2, x1:x2, :]
    height, width, _ = np_img.shape
    for i in range(0, height, patch_size):
        for j in range(0, width, patch_size):
            patch = np_img[i:i + patch_size, j:j + patch_size]
            yield patch, i, j

In [ ]:
image_path = "patient_100/patient_100_node_3.tif"
patch_image = patch_image(image_path, patch_size=224, x1=55000, y1=35000, step_size=224*50)
dataset = []
loaded_model = tf.keras.layers.TFSMLayer("/Users/cilem/.cache/huggingface/hub/models--google--path-foundation/snapshots/fd6a835ceaae15be80db6abd8dcfeb86a9287e72", call_endpoint='serving_default')
infer = loaded_model.call

for i, (patch, x, y) in enumerate(patch_image):
    tensor = tf.cast(tf.expand_dims(np.array(patch), axis=0), tf.float32) / 255.0
    tf.config.optimizer.set_experimental_options({"enable_xla": False})
    embeddings = infer(tf.constant(tensor))
    embedding_vector = embeddings['output_0'].numpy()
    dataset.append({"image_name": f"image_{i}.png",
                    "image": patch,
                    "embedding": embedding_vector})
    if i% 100 == 0:
        plt.imshow(patch)
        plt.show()

In [ ]:
print(len(dataset))
import pickle   
mode = "node_3_1"
with open(f"{mode}_dataset.pkl", "wb") as f:
    pickle.dump(dataset, f)